In [ ]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import openpyxl
from pathlib import Path
import numpy as np
from gensim.models import KeyedVectors

df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/models/last_data.xlsx") 

In [ ]:
df = df[["tweet","label"]]
sentences = df["tweet"].values
y = df["label"].values

training_texts, test_texts, training_labels, test_labels = train_test_split(sentences, y, test_size=0.25, random_state=1000)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)
train,val = train_test_split(train, test_size=0.2)

In [ ]:
!pip install transformers
import transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import torch

Download the pre-trained Turkish BERT Model

In [ ]:
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-128k-uncased', do_lower_case=True)
sentences = df.tweet.values
max_len = 50

Downloading:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/386 [00:00<?, ?B/s]

In [ ]:
from transformers import BertModel

model = BertModel.from_pretrained('dbmdz/bert-base-turkish-128k-uncased',output_hidden_states=True)
model.eval()

In [ ]:
marked_text = training_texts[0]

Last 4 Layer of BERT + CNN

In [ ]:
import torch

In [ ]:
def last_4_emb(text):
  tokenized_text = tokenizer.tokenize(text)
  # Map the token strings to their vocabulary indeces.
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  # Display the words with their indeces.
  
  tokens_tensor = torch.tensor([indexed_tokens])
  
  with torch.no_grad():
    outputs = model(tokens_tensor)
    last_hidden_state = outputs[0]
    word_embed_1 = last_hidden_state
    hidden_states = outputs[2]
    # initial embeddings can be taken from 0th layer of hidden states
    word_embed_2 = hidden_states[0]
    # sum of all hidden states
    word_embed_3 = torch.stack(hidden_states).sum(0)
    # sum of second to last layer
    word_embed_4 = torch.stack(hidden_states[2:]).sum(0)
    # sum of last four layer
    word_embed_5 = torch.stack(hidden_states[-4:]).sum(0)
    # concatenate last four layers
    word_embed_6 = torch.cat([hidden_states[i] for i in [-1,-2,-3,-4]], dim=-1)

    sentence_emb = np.array(word_embed_6[0].sum(0))

    return sentence_emb
  

  


In [ ]:
train_emb = []

for tweet in train.tweet:
  train_emb.append(last_4_emb(tweet))

In [ ]:
val_emb = []

for tweet in val.tweet:
  val_emb.append(last_4_emb(tweet))

In [ ]:
test_emb = []

for tweet in test.tweet:
  test_emb.append(last_4_emb(tweet))

In [ ]:
y_train = train.label
y_test = test.label
y_val = val.label

In [ ]:
x_train = np.array(train_emb)
x_val = np.array(val_emb)

X_train = x_train.reshape(-1, 3072, 1)
X_val = x_val.reshape(-1, 3072, 1)

Training CNN with the BERT Embeddings

In [ ]:
import keras
from keras import models
from keras.models import Sequential
from keras import layers
from keras.layers import Embedding, Bidirectional, Dense, LSTM, Conv1D, MaxPooling1D, Flatten, Reshape, TimeDistributed
from keras import optimizers

model1 = Sequential()

model1.add(Conv1D(256, 3, activation='relu')) # input_shape = (768*3,1)

model1.add(Conv1D(128, 3,  activation='relu'))

model1.add(Conv1D(128, 3,  activation='relu'))


model1.add(MaxPooling1D(5))

# flat
model1.add(Flatten())

model1.add(Dense(3, activation='softmax'))

model1.compile(loss='sparse_categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

history  = model1.fit(np.array(X_train), np.array(y_train),
 epochs= 30,
 batch_size = 200,
 validation_data = (np.array(X_val), np.array(y_val))
)


In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/models/BERT_last4_CNN.pt'
torch.save(model1, path)

INFO:tensorflow:Assets written to: ram://ced7fb2a-694a-4724-80dc-fbec6eae01de/assets


In [ ]:
X_test = np.array(test_emb)
X_test = X_test.reshape((-1, 768*4, 1))
predict_x=model1.predict(X_test) 
prediction=np.argmax(predict_x,axis=1)

In [ ]:
from sklearn import metrics

print(metrics.confusion_matrix(y_test, prediction))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, prediction, digits=3))

[[ 74  71   8]
 [ 38 187  34]
 [ 10  33  68]]
              precision    recall  f1-score   support

           0      0.607     0.484     0.538       153
           1      0.643     0.722     0.680       259
           2      0.618     0.613     0.615       111

    accuracy                          0.629       523
   macro avg      0.622     0.606     0.611       523
weighted avg      0.627     0.629     0.625       523



Last Layer of BERT + CNN

In [ ]:
def last_emb(text):
  tokenized_text = tokenizer.tokenize(text)
  # Map the token strings to their vocabulary indeces.
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  # Display the words with their indeces.
  
  tokens_tensor = torch.tensor([indexed_tokens])
  
  with torch.no_grad():
    outputs = model(tokens_tensor)
    last_hidden_state = outputs[0]
    sentence_emb = np.array(last_hidden_state[0].sum(0))

    return sentence_emb

In [ ]:
train_emb = []

for tweet in train.tweet:
  train_emb.append(last_emb(tweet))

In [ ]:
val_emb = []

for tweet in val.tweet:
  val_emb.append(last_emb(tweet))

In [ ]:
test_emb = []

for tweet in test.tweet:
  test_emb.append(last_emb(tweet))

In [ ]:
y_train = train.label
y_test = test.label
y_val = val.label

x_train = np.array(train_emb)
x_val = np.array(val_emb)

X_train = x_train.reshape(-1, 768, 1)
X_val = x_val.reshape(-1, 768, 1)

Training the CNN with the embeddings from BERT

In [ ]:
import keras
from keras import models
from keras.models import Sequential
from keras import layers
from keras.layers import Embedding, Bidirectional, Dense, LSTM, Conv1D, MaxPooling1D, Flatten, Reshape, TimeDistributed
from keras import optimizers

model2 = Sequential()

model2.add(Conv1D(256, 3, activation='relu')) # input_shape = (768,1)

model2.add(Conv1D(128, 3,  activation='relu'))

model2.add(Conv1D(128, 3,  activation='relu'))



model2.add(MaxPooling1D(5))

# flat
model2.add(Flatten())

model2.add(Dense(3, activation='softmax'))

model2.compile(loss='sparse_categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

history  = model2.fit(np.array(X_train), np.array(y_train),
 epochs= 30,
 batch_size = 200,
 validation_data = (np.array(X_val), np.array(y_val))
)

Epoch 1/30
9/9 [==============================] - 2s 124ms/step - loss: 2.1179 - accuracy: 0.3980 - val_loss: 1.0575 - val_accuracy: 0.4737
Epoch 2/30
9/9 [==============================] - 1s 71ms/step - loss: 1.0464 - accuracy: 0.4333 - val_loss: 1.0428 - val_accuracy: 0.4737
Epoch 3/30
9/9 [==============================] - 1s 71ms/step - loss: 1.0142 - accuracy: 0.5057 - val_loss: 1.0395 - val_accuracy: 0.4737
Epoch 4/30
9/9 [==============================] - 1s 72ms/step - loss: 1.0034 - accuracy: 0.4985 - val_loss: 1.0754 - val_accuracy: 0.4737
Epoch 5/30
9/9 [==============================] - 1s 70ms/step - loss: 0.9610 - accuracy: 0.5554 - val_loss: 0.9445 - val_accuracy: 0.5574
Epoch 6/30
9/9 [==============================] - 1s 70ms/step - loss: 0.8694 - accuracy: 0.5937 - val_loss: 0.8921 - val_accuracy: 0.5766
Epoch 7/30
9/9 [==============================] - 1s 71ms/step - loss: 0.7954 - accuracy: 0.6338 - val_loss: 0.8491 - val_accuracy: 0.5957
Epoch 8/30
9/9 [==========

In [ ]:
X_test = np.array(test_emb)
X_test = X_test.reshape((-1, 768, 1))
predict_x=model2.predict(X_test) 
prediction=np.argmax(predict_x,axis=1)

In [ ]:
from sklearn import metrics

print(metrics.confusion_matrix(y_test, prediction))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, prediction, digits=3))

[[ 77  68   8]
 [ 57 176  26]
 [ 17  28  66]]
              precision    recall  f1-score   support

           0      0.510     0.503     0.507       153
           1      0.647     0.680     0.663       259
           2      0.660     0.595     0.626       111

    accuracy                          0.610       523
   macro avg      0.606     0.592     0.598       523
weighted avg      0.610     0.610     0.609       523

